In [17]:
from asyncio import wait
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException


In [18]:
#!!! Firefox about:config -> browser.download.alwaysOpenPanel = false !!!

#Settings

file_paths = [
'/Volumes/PortableSSD/AI/Projekt OF/Person 1/original.jpg',

]
prompt_text = [
    'A photo of a woman img, hiking, photo-realistic, finely-detailed, sharp focus, clear and precise, natural lighting, realistic skin tones, natural-looking materials, realistic environment',
    'A photo of a woman img, skiing, photo-realistic, finely-detailed, sharp focus, clear and precise, natural lighting, realistic skin tones, natural-looking materials, realistic environment',
]
negative_prompt_text = 'unrealistic, cartoon, blurry, abstract elements, fantasy, unrealistic backgrounds, bad anatomy, bad hands, text, error, missing fingers, cropped, worst quality, low quality, normal quality, jpeg artifacts'
iterations = 20
sample_steps_text = '35'
style_strength_text = '35'


In [19]:

# Joining paths
all_file_paths = '\n'.join(file_paths)

In [20]:

# Firefox WebDriver initialisieren
driver = webdriver.Firefox()

# Öffnen einer Webseite
driver.get("http://127.0.0.1:7860/")

# Warten, um sicherzustellen, dass die Seite geladen ist
time.sleep(2)

In [21]:
prompt_counter = 0

def inputs():

    global prompt_counter
   
    # Upload pictures
    upload_element = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[1]/button/input')
    upload_element.send_keys(all_file_paths)

    # Type in prompt
    prompt = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[4]/div[1]/label/textarea')
    prompt.clear()
    prompt.send_keys(prompt_text[prompt_counter % len(prompt_text)])
    prompt_counter += 1
    
    # Open advanced options
    advanced_options_button = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[5]/button/span[2]')
    advanced_options_button.click()
    
    # Type in negative prompt
    negative_prompt = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[5]/div[2]/div/div/div[1]/label/textarea')
    negative_prompt.clear()
    negative_prompt.send_keys(negative_prompt_text)
    
    # Type in sample steps
    sample_steps = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[5]/div[2]/div/div/div[2]/div[2]/div/input')
    sample_steps.clear()
    sample_steps.send_keys(sample_steps_text)

    # Type in style-strength
    style_strength = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[5]/div[2]/div/div/div[2]/div[2]/div/input')
    style_strength.clear()
    style_strength.send_keys(style_strength_text)
    
def start():

    print("Start iteration")
    submit_button = driver.find_element(By.XPATH, '//*[@id="component-12"]')
    submit_button.click()



In [22]:
def get_percent_value(driver):
    
    element = driver.find_element(By.CSS_SELECTOR, "div.progress-level-inner.svelte-1occ011")
    percent_text = element.text
    # Convert the text to a number and remove the percent sign
    percent_value = int(float(percent_text.rstrip('%')))
    return percent_value

def check_percent():

    while True:
        try:
            percent_value = get_percent_value(driver)
            print(f"Current progress: {percent_value}%")
            if percent_value == 100:
                print("Percentage reached")
                break
        except NoSuchElementException:
            print("Generating done")
            break
        time.sleep(1)

In [23]:
def check_pictures():
    
    wait = WebDriverWait(driver, 4) 
    # Checking if both picutres are open 
    button_xpath = '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[2]/div[1]/button/div[1]/div[2]/button/div/svg'  
    try:
        button = wait.until(EC.visibility_of_element_located((By.XPATH, button_xpath)))
        button.click()
    except:
        print("Both pictures shown.")

def download_pictures():

    print("Download pictures")

    picture_1 = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[2]/div[1]/div[2]/div/button[1]/img')
    picture_1.click()

    download_picture = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[2]/div[1]/button/div[1]/div[1]/button')
    download_picture.click()

    close_picture = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[2]/div[1]/button/div[1]/div[2]/button')
    close_picture.click()

    time.sleep(2)

    picture_2 = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[2]/div[1]/div[2]/div/button[2]/img')
    picture_2.click()

    download_picture = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[2]/div[1]/button/div[1]/div[1]/button')
    download_picture.click()

In [24]:


for _ in range(iterations):

    driver.refresh()
    time.sleep(3) # buffer
    inputs()

    start()
    time.sleep(15) # buffer

    check_percent()


    print("Buffer after generating")
    time.sleep(30) # buffer

    success = False
    max_retry_count = 5
    retry_count = 0
    while not success and retry_count < max_retry_count:
        try:

            check_pictures()

            download_pictures()

            time.sleep(2)

            # If download folder still open
            container = driver.find_element(By.XPATH, '/html/body/gradio-app/div/div/div[1]/div/div/div[4]/div[1]/div[4]/div[1]/label/textarea')
            container.click()

            print("Iteration done")
            success = True
            
        except ElementClickInterceptedException:
            print("ElementClickInterceptedException, trying again.")
            time.sleep(60)
            retry_count += 1

        except (NoSuchElementException, TimeoutException):
            print("NoSuchElementException, TimeoutException, trying again.")
            time.sleep(60) 
            retry_count += 1

    if retry_count >= max_retry_count:
        print("Failed to download pictures, starting new iteration.")

            
# Schließen des Browsers am Ende aller Iterationen
driver.quit()


Start iteration
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 0%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 2%
Current progress: 5%
Current progress: 5%
Current progress: 5%
Current progress: 5%
Current progress: 5%
Current progress: 

KeyboardInterrupt: 